In [1]:
### Stock Valuation Program 
### Goal: create python algorithm to webscrape financials and historical data to formulate multiple valuation models and map the results on a football field chart

In [2]:
import pandas as pd
import requests
import numpy as np
import math
import plotly.graph_objects as go
import plotly.express as px
import time

In [3]:
from sec_api import XbrlApi
import yfinance as yf

In [4]:
headers = {'User-Agent': "ericbstratford@gmail.com"}


tickers_cik = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)
tickers_cik = pd.json_normalize(pd.json_normalize(tickers_cik.json(), max_level=0).values[0])
tickers_cik["cik_str"] = tickers_cik["cik_str"].astype(str).str.zfill(10)
tickers_cik.set_index("ticker",inplace=True)

# response = requests.get("https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/Assets.json", headers=headers)
# assets_timeserie = pd.json_normalize(response.json()["units"]["USD"])
# assets_timeserie["filed"] = pd.to_datetime(assets_timeserie["filed"])
# assets_timeserie = assets_timeserie.sort_values("end")

In [5]:
def get_tags(ticker):
    cik_str = "CIK"+str(tickers_cik.cik_str.loc[ticker])
    title = str(tickers_cik.title.loc[ticker])
    request = "https://data.sec.gov/api/xbrl/companyfacts/"+cik_str+".json"
    res = requests.get(request, headers = headers).json()
    tags = list(res['facts']['us-gaap'].keys())
    return tags
# get_tags("PEP")

In [6]:
def get_fact(ticker, tag):
    request = "https://data.sec.gov/api/xbrl/companyconcept/CIK"+str(tickers_cik.cik_str.loc[ticker])+"/us-gaap/"+tag+".json"
    response = requests.get(request, headers = headers)
    df = pd.json_normalize(response.json()["units"]['USD'])
    df["filed"] = pd.to_datetime(df["filed"])
    df["end"] = pd.to_datetime(df["end"])
    df.sort_values("end", inplace=True)
    df = df.groupby("end")[['val']].mean().rename(columns={"val":tag})
    return df

def get_other(ticker, tag, unit):
    request = "https://data.sec.gov/api/xbrl/companyconcept/CIK"+str(tickers_cik.cik_str.loc[ticker])+"/us-gaap/"+tag+".json"
    response = requests.get(request, headers = headers)
    df = pd.json_normalize(response.json()["units"][unit])
    df["filed"] = pd.to_datetime(df["filed"])
    df["end"] = pd.to_datetime(df["end"])
    df.sort_values("end", inplace=True)
    df = df.groupby("end")[['val']].mean().rename(columns={"val":tag})
    return df

def get_market_val(ticker, tag):
    market_data = yf.Ticker(ticker)
    out = market_data.info[tag]
    return out

In [132]:
# yf.Ticker("VZ").info.keys()
# yf.Ticker("SPY").history(period='5y')
# yf.Ticker("PEP").get_cash_flow()

In [133]:
# [UNDER CONSTRUCTION]
# ticker = "WM"

# months_in_year = 12
# period = '5y'

# mkt = yf.Ticker("SPY").history(period=period)
# mkt = mkt[pd.to_datetime(mkt.index).day==1]
# mkt = mkt[['Close']].rename(columns={'Close':'SPY_Close'})
# mkt['SPY_MR'] = mkt['SPY_Close'].pct_change()
# mkt['SPY_AR'] = (1 + mkt['SPY_MR']).prod() ** (months_in_year / len(mkt)) - 1
# mkt['SPY_CR'] = mkt['SPY_MR'].cumsum()
# mkt

# add = yf.Ticker(ticker).history(period=period)
# add = add[pd.to_datetime(add.index).day==1]
# rename = ticker+"_Close"
# mon_rt = ticker+"_MR"
# ann_rt = ticker+"_AR"
# cum_rt = ticker+"_CR"
# add = add[['Close']].rename(columns={'Close':rename})
# add[mon_rt] = add[rename].pct_change()
# add[ann_rt] = (1 + add[mon_rt]).prod() ** (months_in_year / len(add)) - 1
# add[cum_rt] = add[mon_rt].cumsum()

# df = pd.concat([mkt, add], axis=1)
# df['MR_Diff'] = df[mon_rt] - df['SPY_MR']
# df['AR_Diff'] = df[ann_rt] - df['SPY_AR']
# df['CR_Diff'] = df[cum_rt] - df['SPY_CR']
# df['AR_Pdiff'] = (df[ann_rt] - df['SPY_AR'])/df['SPY_AR']
# df

In [12]:
def yf_dcf(ticker):
    total_debt = get_market_val(ticker, "totalDebt")
    shares_outstanding = get_market_val(ticker, "sharesOutstanding")
    current_price = get_market_val(ticker, "currentPrice")
    market_enterprise_value = get_market_val(ticker, "enterpriseValue")
    market_equity_value = market_enterprise_value - total_debt
    market_intrinsic_value = market_equity_value / shares_outstanding
    print("Current Price:", current_price)
    print("Market Intrinsic Value:", market_intrinsic_value)
    if market_intrinsic_value > current_price*1.10:
        return "Strong Buy"
    elif market_intrinsic_value < current_price*0.9:
        return "Strong Sell"
    elif market_intrinsic_value > current_price*1.02:
        return "Buy"
    elif market_intrinsic_value < current_price*0.98:
        return "Sell"
    else:
        return "Hold"
# yf_dcf("COST")

In [131]:
def discounted_cash_flow(ticker):
    facts = ["OperatingIncomeLoss", "AccountsPayableCurrent", "AccountsReceivableNetCurrent", 
             ("EffectiveIncomeTaxRateContinuingOperations", "pure"), 
            ]
    da1 = ["DepreciationAndAmortization"]
    da2 = ["Depreciation", "AmortizationOfIntangibleAssets"]
    ce1 = ["PaymentsToAcquirePropertyPlantAndEquipment"]
    ce2 = ["PaymentsToAcquireProductiveAssets"]
    
    tags = get_tags(ticker)
    
#     return yf_dcf(ticker)
    
    master_df = pd.DataFrame()
    for fact in facts:
        try:
            if type(fact)==tuple:
                fact_df = get_other(ticker, fact[0], fact[1])
            else:
                fact_df = get_fact(ticker, fact)
        except:
            print("Unavailable Cash Flow:", fact)
            return yf_dcf(ticker)
        
        if master_df.empty:
            master_df = fact_df
        else:
            master_df = master_df.merge(fact_df, how='left', on='end')
    
    # Depreciation and Amortization
    if da1[0] in tags:
        fact_df = get_fact(ticker, da1[0])
        master_df = master_df.merge(fact_df, how='left', on='end')
    elif da2[0] in tags and da2[1] in tags:
        fact_df = get_fact(ticker, da2[0])
        master_df = master_df.merge(fact_df, how='left', on='end')
        fact_df = get_fact(ticker, da2[1])
        master_df = master_df.merge(fact_df, how='left', on='end')
    else:
        raise ValueError("No Valid Depr&Amor")
    
    if ce1[0] in tags:
        fact_df = get_fact(ticker, ce1[0])
        master_df = master_df.merge(fact_df, how='left', on='end')
    elif ce2[0] in tags:
        fact_df = get_fact(ticker, ce2[0])
        master_df = master_df.merge(fact_df, how='left', on='end')
    else:
        raise ValueError("No Valid CapEx")
        
    
        
    

    # Calculate Operating Cash Flow
    if "DepreciationAndAmortization" not in master_df.columns:
#         master_df['Depreciation'] = master_df['Depreciation'].ffill()/4
        master_df['NonCashExpenses'] = master_df['Depreciation'] + master_df['AmortizationOfIntangibleAssets']
    else:
        master_df['NonCashExpenses'] = master_df['DepreciationAndAmortization']
        
    master_df = master_df.fillna(0)
#     master_df = master_df.ffill().bfill()
    
    master_df['AccRecNet'] = master_df['AccountsReceivableNetCurrent']
    master_df['AccPayChg'] = master_df['AccountsPayableCurrent'].diff()
    master_df['AccRecChg'] = master_df['AccRecNet'].diff()
    master_df['NetWorkCapChg'] = master_df['AccPayChg'] + master_df['AccRecChg']
    master_df['Taxes'] = master_df['OperatingIncomeLoss'] * master_df['EffectiveIncomeTaxRateContinuingOperations']
    master_df['OCF'] = master_df['OperatingIncomeLoss'] + master_df['NonCashExpenses'] - master_df['NetWorkCapChg'] - master_df['Taxes']

    # Calculate Free Cash Flow
    if "PaymentsToAcquirePropertyPlantAndEquipment" in master_df.columns:
        master_df['CapEx'] = master_df['PaymentsToAcquirePropertyPlantAndEquipment']
    else:
        master_df['CapEx'] = master_df['PaymentsToAcquireProductiveAssets']
    master_df['FCF'] = master_df['OCF'] - master_df['CapEx']
    
    master_df.dropna(how='any', inplace=True)
    master_df = master_df[master_df.index >= pd.to_datetime("2014-01-01")]
#     return master_df
#     return master_df[master_df.isna().any(axis=1)]
    
    
    # Project Future Free Cash Flows
    df = master_df[['FCF']]
    df.reset_index(inplace=True)
    def calculate_cagr(first, last, periods):
        return (last / first) ** (1 / periods) - 1
    first_fcf = df['FCF'].iloc[0]
    last_fcf = df['FCF'].iloc[-1]
    n_periods = len(df) - 1
    cagr = calculate_cagr(first_fcf, last_fcf, n_periods)
    forecast_period = 10 * 4  # 10 years in quarters
    future_dates = pd.date_range(start=df['end'].iloc[-1], periods=forecast_period + 1, freq='QE')
    future_fcfs = [df['FCF'].iloc[-1] * (1 + cagr) ** i for i in range(1, forecast_period + 1)]
    future_df = pd.DataFrame({'end': future_dates[1:], 'FCF': future_fcfs})
#     df_projected = pd.concat([df, future_df]).reset_index(drop=True)
    df_projected = future_df
#     return df_projected
    
    
    # Query Market Values
    market_cap = get_market_val(ticker, "marketCap")
    total_debt = get_market_val(ticker, "totalDebt")
    shares_outstanding = get_market_val(ticker, "sharesOutstanding")
    current_price = get_market_val(ticker, "currentPrice")
    market_enterprise_value = get_market_val(ticker, "enterpriseValue")
    forward_pe = get_market_val(ticker, "forwardPE")
    ebitda = get_market_val(ticker, "ebitda")
    interest_expense = get_fact(ticker, "InterestExpense").iloc[-1,0]
    
    
    # WACC - Weighted Average Cost of Capital
    risk_free_rate = 0.03  # General Risk-free rate
    market_risk_premium = 0.06  # General Market risk premium
    tax_rate = 0.21  # Corporate tax rate

    beta = get_market_val(ticker,"beta")

    cost_of_equity = risk_free_rate + beta * market_risk_premium
    cost_of_debt = (interest_expense / total_debt) * (1 - tax_rate)

    market_value_equity = market_cap
    market_value_debt = total_debt

    equity_weight = market_value_equity / (market_value_equity + market_value_debt)
    debt_weight = market_value_debt / (market_value_equity + market_value_debt)

    wacc = equity_weight * cost_of_equity + debt_weight * cost_of_debt
#     print(wacc)
    q_wacc = (1 + wacc) ** (1/4) - 1 # Transform WACC to Quarterly 

    
    discount_rate = q_wacc
    print("wacc:", wacc)
    

    g = cagr
    print("g:", g)
    FCF_terminal_qtr = df_projected['FCF'].iloc[-1]
    FCF_terminal_yr = FCF_terminal_qtr*4
    terminal_value = FCF_terminal_yr * (1 + g) / (wacc - g)
    
#     emm_terminal_value = ebitda*10
#     print(pge_terminal_value, pgn_terminal_value, emm_terminal_value)
#     terminal_value = emm_terminal_value

    
    # Present-Value
    # Create an array to store the discounted cash flows
    discounted_cash_flows = []
    # Discount each projected cash flow to its present value
    for i, cash_flow in enumerate(df_projected['FCF']):
        discounted_cash_flow = cash_flow / (1 + discount_rate) ** (i + 1)  # Discounted to present value
        discounted_cash_flows.append(discounted_cash_flow)
    # Discount the terminal value to its present value
    discounted_terminal_value = terminal_value / ((1 + discount_rate) ** len(df_projected))
    pv_fcf = np.sum(discounted_cash_flows)
    pv_tv = discounted_terminal_value
    enterprise_value = pv_fcf + pv_tv
    
    # Intrinsic Value
    equity_value = enterprise_value - total_debt
    intrinsic_value_per_share = equity_value / shares_outstanding
    print("Current Price:", current_price)
    print("DCF Intrinsic Value:", intrinsic_value_per_share)
    
    
    # Market DCF
    market_equity_value = market_enterprise_value - total_debt
    market_intrinsic_value = market_equity_value / shares_outstanding
    print("YFin Intrinsic Value:", market_intrinsic_value)

    return

discounted_cash_flow("AAPL")

wacc: 0.10319949903971426
g: 0.013101790646619094
Current Price: 189.84
DCF Intrinsic Value: 218.74518546203115
YFin Intrinsic Value: 185.34064833455506
